In [ ]:
import os
import pandas as pd
from ultralytics import YOLO, settings

WINDOWS_DRIVE_DATA = "/mnt/f/COCO2017_Data"
os.makedirs(WINDOWS_DRIVE_DATA, exist_ok=True)
settings.update({'datasets_dir': WINDOWS_DRIVE_DATA})

MODEL_CONFIG = "/home/diy-hus/PCBNet_NEW/pcbnet-v2.yaml"
DATA_YAML = "coco.yaml"
PROJECT = "./runs_pcbnet_v2_coco"
NAME = "pretrain_pcbnet_v2_scratch"

try:
    model = YOLO(MODEL_CONFIG)
except Exception as e:
    print(f"Error loading config: {e}")
    exit()

def print_all_metrics(trainer):
    current_epoch = trainer.epoch + 1
    print(f"\n>>> Full Metrics Report for Epoch {current_epoch} <<<")
    
    if hasattr(trainer, 'metrics'):
        for key, value in trainer.metrics.items():
            print(f"{key:<25}: {value:.5f}")
    
    if hasattr(trainer, 'tloss') and trainer.tloss is not None:
        if trainer.tloss.numel() == 1:
            print(f"{'Training Loss':<25}: {trainer.tloss.item():.5f}")
        else:
            loss_values = trainer.tloss.tolist()
            loss_str = ", ".join([f"{x:.5f}" for x in loss_values])
            print(f"{'Training Losses':<25}: [{loss_str}] (Box, Cls, DFL)")
            
    print("-" * 50)

model.add_callback("on_fit_epoch_end", print_all_metrics)

print("Starting training on WSL...")

try:
    results = model.train(
        data=DATA_YAML,
        epochs=100,
        batch=16,
        imgsz=640,
        workers=0,
        amp=False,
        patience=10,
        optimizer='AdamW',
        lr0=0.001,
        cos_lr=True,
        device=0,
        project=PROJECT,
        name=NAME,
        exist_ok=True,
        plots=True,
        save=True,
        save_period=-1
    )
except Exception as e:
    print(f"\nTRAINING FAILED: {e}")
    print("Suggestion: Try reducing 'batch' size if facing OOM errors.")

csv_path = os.path.join(PROJECT, NAME, "results.csv")
if os.path.exists(csv_path):
    print(f"\nMetrics saved to CSV: {csv_path}")
    try:
        df = pd.read_csv(csv_path)
        df.columns = df.columns.str.strip()
        print("\nFinal Epoch Metrics:")
        print(df.tail(1).T)
    except Exception as e:
        print(f"Could not read CSV file: {e}")